<a href="https://colab.research.google.com/github/Emillka/Python_learn_projects/blob/main/fin_website_links.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import pandas as pd
from bs4 import BeautifulSoup
from requests import get  # Dodano import

def pobierz_zawartosc_html(url):
    response = get(url)
    return response.text if response.status_code == 200 else None

def znajdz_odnosniki_do_strony(html):
    if html is None:
        return set()

    soup = BeautifulSoup(html, 'html.parser')
    odnosniki = soup.find_all('a', href=re.compile(r'^https?://(?:www\.)?nazwa-domeny\.pl/'))

    return {odnosnik['href'] for odnosnik in odnosniki}

if __name__ == "__main__":
    nazwa_pliku = "tutaj wkleić ścieżkę do pliku excel z adresami url do sprawdzenia"
    df = pd.read_excel(nazwa_pliku)

    if 'url' in df.columns:
        adresy_url = df['url'].dropna().tolist()
        if not adresy_url:
            print("Nie znaleziono adresów URL w pliku.")
        else:
            print("Sprawdzanie odnośników dla podanych adresów URL:")
            for url in adresy_url:
                zawartosc_html = pobierz_zawartosc_html(url)
                if zawartosc_html is not None:  # Dodano sprawdzenie, czy pobranie zawartości było udane
                    odnosniki = znajdz_odnosniki_do_strony(zawartosc_html)
                    if odnosniki:
                        print(f"Adres URL: {url}")
                        for odnosnik in odnosniki:
                            print(odnosnik)
                    else:
                        print(f"Adres URL: {url} - Brak odnośników do witryny lub jej subdomen.")
                else:
                    print(f"Nie udało się pobrać zawartości HTML dla adresu URL: {url}")
    else:
        print("Nie znaleziono kolumny 'url' w pliku XLSX.")
